<a href="https://colab.research.google.com/github/nssn96/ML_Neural-networks/blob/main/ML_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML Assignment 2 - Neural Networks


1.   Author : Surya Narayanan Nadhamuni Suresh
2.   UTA ID : 1001877873



In [69]:
#import lines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [70]:
#References Used
#https://towardsdatascience.com/https-medium-com-piotr-skalski92-deep-dive-into-deep-networks-math-17660bc376ba
#https://towardsdatascience.com/lets-code-a-neural-network-in-plain-numpy-ae7e74410795

# This is the Layer class

class Layer:

  def initial_w_bias(self,nn):
    
    #random seed initiation
    np.random.seed(90)
    n_layers = len(nn)
    param = dict()

    for index,layer in enumerate(nn):
      layer_num = index+1
      lyr_inputsize = layer['d_input']
      lyr_outputsize = layer['d_output']

      #creating the values for the w and bias matrix
      param['w'+str(layer_num)] = np.random.randn(lyr_outputsize,lyr_inputsize) * 0.1
      param['b'+str(layer_num)] = np.random.randn(lyr_outputsize,1) * 0.1


  def forward(self):
    pass


  def backward(self):
    pass



In [71]:
# This is the Linear layer class

class Linearlayer(Layer):

  def forward_single(self):
    pass
  
  def forward(self):
    return param
  
  def backward_single(self):
    pass
  
  def backward(self):
    pass
  

In [74]:
#This is the class for the sigmoid function

class sigmoid_activation(Layer):

  def sigmoid_forwardprop(self,Z):
   return 1/(1+np.exp(-Z))
  
  def sigmoid_backprop(self,dA,z):
    value = self.sigmoid(z)
    return dA * value * (1-value)
  

In [ ]:
#This is the class for the hyperbolic tangent functions

class htangent_activation(Layer):

  def tanh_forwardprop(self,z):
    #2/(1 + e-2x) - 1
    return (2/(1+np.exp(-2*z)) -1)
  
  def tanh_backprop(self,dA,z):
    value = self.tanh_forwardprop(z)
    return dA * value * (1-value)


In [ ]:
#This is the class for the softmax function

class softmax_activation(Layer):

  def softmax_forwardprop(self,z):
    return np.exp(z) / sum(np.exp(z))

  
  def softmax_backprop(self,dA,z):
    value = self.softmax_forwardprop(z)
    return dA * value * (1-value)

  

In [72]:
# I will be providing the a list of dictionary that contains the parameters for the Neural network( ie the number of layers, number of nodes and activation function)
#d_input--> size of the input vector to the layer ie n(l-1) where l is layer
#d_ouput--> size of the activation vector at the output of the layer ie n(l)
nn = [
      {'d_input':2 ,'d_output':3, 'activation': 'tanh' } ,
       {'d_input':3 ,'d_output':1, 'activation': 'sigmoid' }
]

print(nn)

[{'d_input': 2, 'd_output': 3, 'activation': 'tanh'}, {'d_input': 3, 'd_output': 1, 'activation': 'sigmoid'}]


In [73]:
#testing
net = Linearlayer()
param = net.initial_w_bias(nn)
#print(net.forward())

{'w1': array([[-0.02244867, -0.02275239],
       [-0.05913989, -0.07992277],
       [ 0.11901095, -0.13658949]]), 'b1': array([[ 0.07209559],
       [-0.13255618],
       [ 0.18569316]]), 'w2': array([[ 0.12471803, -0.13542584, -0.07404735]]), 'b2': array([[0.02340754]])}
